In [8]:
import torch
import lightning
import os
from matplotlib import pyplot as plt
import SimpleITK as sitk


In [9]:
path = '/share/sda/nurenzhaksylyk/segaorta_resampled'

In [10]:
patients = os.listdir(path)
len(patients)

56

In [13]:
pat = sitk.ReadImage(os.path.join(path, patients[0], (patients[0]+'_ct.nrrd')))

In [15]:
a = sitk.GetArrayFromImage(pat)